# Comp1-1传统组学特征提取

主要适配于传统组学的建模和刻画。典型的应用场景探究rad_score最最终临床诊断的作用。

数据的一般形式为(具体文件,文件夹名可以不同)：
1. `images`文件夹，存放研究对象所有的CT、MRI等数据。
2. `masks`文件夹, 存放手工（Manuelly）勾画的ROI区域。与images文件夹的文件意义对应。

## Onekey步骤

1. 数据校验，检查数据格式是否正确。
2. 组学特征提取，如果第一步检查数据通过，则提取对应数据的特征。
5. 查看一些统计信息，检查数据时候存在异常点。
6. 正则化，将数据变化到服从 N~(0, 1)。
7. 通过相关系数，例如spearman、person等筛选出特征。


## 一、数据校验
首先需要检查诊断数据，如果显示`检查通过！`择可以正常运行之后的，否则请根据提示调整数据。

**注意**：这里要求images和masks文件夹中的文件名必须一一对应。e.g. `1.nii.gz`为images中的一个文件，在masks文件夹必须也存在一个`1.nii.gz`文件。

当然也可以使用自定义的函数，获取解析数据。

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
from pathlib import Path
from onekey_algo.custom.components.Radiology import diagnose_3d_image_mask_settings, get_image_mask_from_dir
from onekey_algo import OnekeyDS as okds

os.makedirs('img', exist_ok=True)
# 设置数据目录
# mydir = r'你自己数据的路径'
mydir = okds.ct

# 生成images和masks对，一对一的关系。也可以自定义替换。
images, masks = get_image_mask_from_dir(mydir, images='images', masks='masks')

# 自定义获取images和masks数据的方法，下面的例子为，每个样本一个文件夹，图像是以im.nii结尾，mask是以seg.nii结尾。
# def get_images_mask(mydir):
#     images = []
#     masks = []
#     for root, dirs, files in os.walk(mydir):
#         for f in files:
#             if f.endswith('im.nii'):
#                 images.append(os.path.join(root, f))
#             if f.endswith('seg.nii'):
#                 masks.append(os.path.join(root, f))
#     return images, masks
# images, masks = get_images_mask(mydir)

diagnose_3d_image_mask_settings(images, masks)
print(f'获取到{len(images)}个样本。')

# 传统组学特征

使用pyradiomics提取传统组学特征，正常这块不需要修改，下面是具体的Onekey封装的接口。

```python
def extract(self, images: Union[str, List[str]], 
            masks: Union[str, List[str]], labels: Union[int, List[int]] = 1, settings=None)
"""
    * images: List结构，待提取的图像列表。
    * masks: List结构，待提取的图像对应的mask，与Images必须一一对应。
    * labels: 提取标注为什么标签的特征。默认为提取label=1的。
    * settings: 其他提取特征的参数。默认为None。

"""
```

```python
def get_label_data_frame(self, label: int = 1, column_names=None, images='images', masks='labels')
"""
    * label: 获取对应label的特征。
    * columns_names: 默认为None，使用程序设定的列名即可。
"""
```

In [ ]:
from onekey_algo.custom.Manager import onekey_show
# 特征提取视频
onekey_show('传统组学任务|特征提取')

In [ ]:
import warnings
import pandas as pd
 
warnings.filterwarnings("ignore")

from onekey_algo.custom.components.Radiology import ConventionalRadiomics

# 如果要自定义一些特征提取方式，可以使用param_file。
# param_file = r'./custom_settings/exampleCT.yaml'
param_file = None
radiomics = ConventionalRadiomics(param_file, correctMask= True)
radiomics.extract(images, masks)
rad_data = radiomics.get_label_data_frame(label=1)
rad_data.head()

# 特征维度

In [ ]:
rad_data.columns

## 获取到数据的统计信息

1. count，统计样本个数。
2. mean、std, 对应特征的均值、方差
3. min, 25%, 50%, 75%, max，对应特征的最小值，25,50,75分位数，最大值。

In [ ]:
rad_data.describe()

## Z-score

`normalize_df` 为onekey中正则化的API，将数据变化到0均值1方差。正则化的方法为

$column = \frac{column - mean}{std}$

In [ ]:
from onekey_algo.custom.components.comp1 import normalize_df
data = normalize_df(rad_data, not_norm=['ID'])
data.describe()

### 相关系数

计算相关系数的方法有3种可供选择
1. pearson （皮尔逊相关系数）: standard correlation coefficient

2. kendall (肯德尔相关性系数) : Kendall Tau correlation coefficient

3. spearman (斯皮尔曼相关性系数): Spearman rank correlation

三种相关系数参考：https://blog.csdn.net/zmqsdu9001/article/details/82840332

In [ ]:
# pearson_corr = data.corr('pearson')
# kendall_corr = data.corr('kendall')
spearman_corr = data.corr('spearman')

### 特征可视化

特征热图，不同特征之间的两两相关性。

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from onekey_algo.custom.components.comp1 import draw_matrix
plt.figure(figsize=(100.0, 80.0))

# 选择可视化的相关系数
draw_matrix(spearman_corr, annot=True, cmap='YlGnBu', cbar=False)
plt.savefig(f'img/feature_corr.svg', bbox_inches = 'tight')

### 聚类分析

通过修改变量名，可以可视化不同相关系数下的相聚类分析矩阵。

注意：当特征特别多的时候（大于100），尽量不要可视化，否则运行时间会特别长。

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

pp = sns.clustermap(spearman_corr, linewidths=.5, figsize=(50.0, 40.0), cmap='YlGnBu')
plt.setp(pp.ax_heatmap.get_yticklabels(), rotation=0)
plt.savefig(f'img/feature_cluster.svg', bbox_inches = 'tight')

### 特征筛选

根据相关系数，对于相关性比较高的特征（一般文献取corr>0.9），两者保留其一。

```python
def select_feature(corr, threshold: float = 0.9, keep: int = 1, topn=10, verbose=False):
    """
    * corr, 相关系数矩阵。
    * threshold，筛选的相关系数的阈值，大于阈值的两者保留其一（可以根据keep修改，可以是其二...）。默认阈值为0.9
    * keep，可以选择大于相关系数，保留几个，默认只保留一个。
    * topn, 每次去掉多少重复特征。
    * verbose，是否打印日志
    """
```

In [ ]:
from onekey_algo.custom.components.comp1 import select_feature
sel_feature = select_feature(spearman_corr, threshold=0.9, topn=10, verbose=False)
sel_feature

In [ ]:
data[['ID'] + sel_feature].head()

### 过滤特征

通过`sel_feature`过滤出筛选出来的特征。

In [ ]:
sel_data = data[['ID'] + sel_feature]
sel_data.describe()

### 保存特征

保存CSV特征，为了后续做多组学融合使用。

In [ ]:
sel_data.to_csv('rad_features.csv', header=True, index=False)

### 特征读取

为了和后续其他的组件串联，需要特征读取模块

In [ ]:
import pandas
read_data = pandas.read_csv('rad_features.csv', header=0)
read_data.head()